# Интерактивная иерархия организаций с переводом описаний на русский

Этот ноутбук создаёт интерактивный граф (HTML) на основе JSON-файлов:
- `entities_full.json` — список организаций и их атрибутов;  
- `connections.json` — связи между организациями.

Особенности:
- Иерархия сверху вниз (**Top-Down**)  
- Адаптивный дизайн для телефонов (**height = 100vh**)  
- Автоматический перевод `description` на русский через `deep-translator`  
- При клике на узел выводится справка под графом

In [ ]:
!pip install pyvis deep-translator

In [ ]:
import json
from pyvis.network import Network
import textwrap
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='ru')

def translate_text(text):
    if not text or len(text.strip()) < 3:
        return text
    try:
        return translator.translate(text)
    except Exception:
        return text

def wrap_label(label: str, width: int = 18) -> str:
    return "\n".join(textwrap.wrap(label or "", width=width, break_long_words=False, break_on_hyphens=False))

In [ ]:
with open('connections.json', encoding='utf-8') as f:
    connections = json.load(f)
with open('entities_full.json', encoding='utf-8') as f:
    entities = json.load(f)

entity_map = {row['id']: row['value']['attributes'] for row in entities['rows'] if 'attributes' in row['value']}
print(f'Загружено элементов: {len(entity_map)}')

In [ ]:
net = Network(height='100vh', width='100%', bgcolor='#111111', font_color='white', directed=True)
net.set_options('''{
  "layout": {"hierarchical": {"enabled": true, "direction": "UD", "sortMethod": "hubsize", "levelSeparation": 250, "nodeSpacing": 280, "treeSpacing": 350, "shakeTowards": "roots"}},
  "interaction": {"zoomView": true, "dragView": true},
  "physics": {"enabled": false},
  "nodes": {"shape": "dot", "size": 14, "font": {"size": 18, "strokeWidth": 2, "multi": "html"}, "margin": 12},
  "edges": {"arrows": {"to": {"enabled": true, "scaleFactor": 0.6}}, "smooth": {"type": "cubicBezier"}}
}''')

In [ ]:
for eid, attrs in entity_map.items():
    label = attrs.get('label', eid)
    etype = attrs.get('element type', 'Unknown')
    parent = attrs.get('parent organization', '')
    desc = translate_text(attrs.get('description', ''))
    website = attrs.get('website', '')
    location = attrs.get('location', '')
    color = '#4da6ff' if etype == 'Air Force' else '#80ff80' if etype == 'Navy' else '#ffcc00' if etype == 'Army' else '#ff6666'
    html_info = f"""<b>{label}</b><br>Тип: {etype}<br>Подразделение: {parent}<br>Местоположение: {location}<br>Сайт: <a href='{website}' target='_blank'>{website}</a><br><i>Описание (рус): {desc}</i>"""
    net.add_node(eid, label=wrap_label(label), title=html_info, color=color)
print('✅ Узлы добавлены с переводом описаний.')

In [ ]:
added_edges = 0
for row in connections['rows']:
    conn = row['value']
    from_id = conn.get('from_id')
    to_id = conn.get('to_id')
    if from_id in entity_map and to_id in entity_map:
        net.add_edge(from_id, to_id)
        added_edges += 1
print(f'✅ Добавлено связей: {added_edges}')

In [ ]:
custom_html = '''<div id="org_info" style="background:#111;color:white;font-size:16px;padding:10px;min-height:100px;margin-top:10px;"><b>Нажмите на узел, чтобы увидеть описание организации.</b></div><script type="text/javascript">network.on("click", function (params) {if (params.nodes.length > 0) {var nodeId = params.nodes[0];var node = nodes.get(nodeId);document.getElementById("org_info").innerHTML = node.title || "Нет описания.";}});</script>'''
net.write_html('org_hierarchy.html', notebook=False)
with open('org_hierarchy.html', 'r+', encoding='utf-8') as f:
    html = f.read()
    html = html.replace('</body>', custom_html + '\n</body>')
    f.seek(0)
    f.write(html)
    f.truncate()
print('✅ Готово! Файл org_hierarchy.html можно открыть в браузере.')